In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import string
import nltk
nltk.download('stopwords')
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/priya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv("./Data/phrases.csv")
# data=pd.DataFrame(data)
# print type(data)
data = data.dropna()
print data['Emotion'].unique()

['joy' 'fear' 'anger' 'sadness' 'disgust' 'shame' 'guilt']


In [3]:
# print data
def clean_dataset(data):
    translator = string.maketrans('', '')
    for index,row in data.iterrows():
#         print(type(row['Phrase']))
        row['Phrase'] = str(row['Phrase']).replace('[','')
        row['Phrase'] = str(row['Phrase']).replace(']','')
        row['Phrase'] = str(row['Phrase']).strip()
        row['Phrase'] = str(row['Phrase']).translate(translator,string.punctuation)
    return data
data = clean_dataset(data)
data.head()

,Emotion,Phrase
0,joy,On days when I feel closing to my partner and ...
1,fear,Every time I imagine that someone I love or I ...
2,anger,When I had been obviously unjustly treated and...
3,sadness,When I think about the short time that we live...
4,disgust,At a gathering I found myself involuntarily si...


In [5]:
def tokenise(data):
    ## Convert words to lower case and split them    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    
    for index,row in data.iterrows():
        text = str(row['Phrase']).lower().split(' ')
        text = [w.strip() for w in text if not w in stops and len(w) >= 2]
        text = " ".join(text)
        row['Phrase'] = text
    # split the dataset into tokens
    return data
data = tokenise(data)
data.head()

,Emotion,Phrase
0,joy,days feel closing partner friends feel peace a...
1,fear,every time imagine someone love could contact ...
2,anger,obviously unjustly treated possibility elucida...
3,sadness,think short time live relate periods life thin...
4,disgust,gathering found involuntarily sitting next two...


In [6]:
def lemmatization(dataset):
    stemmer = SnowballStemmer('english')
    list_of_words=[]
    for index,row in dataset.iterrows():
        text = str(row['Phrase']).split()
        stemmed_words = [stemmer.stem(word) for word in text]
        list_of_words.append(stemmed_words)
        text = " ".join(stemmed_words)
        row['Phrase'] = text
    data['list_of_words'] = list_of_words
    return dataset
data = lemmatization(data)
data.head()

,Emotion,Phrase,list_of_words
0,joy,day feel close partner friend feel peac also e...,"[day, feel, close, partner, friend, feel, peac..."
1,fear,everi time imagin someon love could contact se...,"[everi, time, imagin, someon, love, could, con..."
2,anger,obvious unjust treat possibl elucid,"[obvious, unjust, treat, possibl, elucid]"
3,sadness,think short time live relat period life think ...,"[think, short, time, live, relat, period, life..."
4,disgust,gather found involuntarili sit next two peopl ...,"[gather, found, involuntarili, sit, next, two,..."
